## Notebook to start making projections over the twenty first century


This notebook is used to make projections of the wind influence on sea level rise in the 21th century. 

Here, the same methods are used as in: DOI 10.1007/s00382-013-1932-4 (A new atmospheric proxy for sea level variability
in the southeastern North Sea: observations and future ensemble projections)


From the regression between zos and vas/uas (cmip6 data) the regression coefficients are used to make predictions into the 21st century (files: nearby_wind_regression_cmip6, timmerman_regression_cmip6, dangendorf_regression_cmip6). Only models are used that perform well according to spectral analysis (file: comparison). 




In [17]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/Users/iriskeizer/Projects/ClimatePhysics/Thesis/Github/Thesis-KNMI/Projections/code')

import import_data as imprt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Projections for only the best models

In [18]:
# Import best models
path_best_models = '/Users/iriskeizer/Projects/ClimatePhysics/Thesis/Data/cmip6/Comparison results/'
models = []

# Source: https://stackabuse.com/reading-and-writing-lists-to-a-file-in-python/
# open file and read the content in a list
with open(path_best_models+'bestmodels.txt', 'r') as filehandle:
    for line in filehandle:
        # remove linebreak which is the last character of the string
        currentPlace = line[:-1]

        # add item to the list
        models.append(currentPlace)

#### Import zos data

In [19]:
# Open data file
zos_119 = imprt.import_cmip6_slh_data(data_type = 'ssp119')
zos_126 = imprt.import_cmip6_slh_data(data_type = 'ssp126')
zos_245 = imprt.import_cmip6_slh_data(data_type = 'ssp245')
zos_370 = imprt.import_cmip6_slh_data(data_type = 'ssp370')
zos_585 = imprt.import_cmip6_slh_data(data_type = 'ssp585')


# Select models
zos_119 = zos_119.where(zos_119.model.isin(models), drop = True)
zos_126 = zos_126.where(zos_126.model.isin(models), drop = True)
zos_245 = zos_245.where(zos_245.model.isin(models), drop = True)
zos_370 = zos_370.where(zos_370.model.isin(models), drop = True)
zos_585 = zos_585.where(zos_585.model.isin(models), drop = True)

#### NearestPoint wind regression model

MSL = constant + coef_u2 x u2 + coef_v2 x v2


What do we need to obtain projections?
- cmip6 data for scenario
- regression results over historical period (from the regression of sea level averaged over all stations)
- whether these regression results are significant

In [20]:
model = 'NearestPoint'

#### Import wind data

In [ ]:
# Open data file
wind_119 = imprt.import_cmip6_wind_data(data_type = 'ssp119')
wind_126 = imprt.import_cmip6_wind_data(data_type = 'ssp126')
wind_245 = imprt.import_cmip6_wind_data(data_type = 'ssp245')
wind_370 = imprt.import_cmip6_wind_data(data_type = 'ssp370')
wind_585 = imprt.import_cmip6_wind_data(data_type = 'ssp585')


# Select models
wind_119_np = wind_119.where(wind_119.model.isin(models), drop = True)
wind_126_np = wind_126.where(wind_126.model.isin(models), drop = True)
wind_245_np = wind_245.where(wind_245.model.isin(models), drop = True)
wind_370_np = wind_370.where(wind_370.model.isin(models), drop = True)
wind_585_np = wind_585.where(wind_585.model.isin(models), drop = True)

#### Import regression coefficients

In [ ]:
import pandas as pd

# Import regression coefficients
path_reg_results = '/Users/iriskeizer/Projects/ClimatePhysics/Thesis/Data/cmip6/Regression results/Projections/'
results_np = pd.read_csv(path_reg_results + f'{model}_results.csv', index_col = 'result')

# Select models
results_np = results_np[models]

In [ ]:
results_np

In [ ]:
results_np

In [ ]:
# Obtain projections of wind contribution to sea level
def wind_contr_proj_np(results, wind_proj):
    
    df = pd.DataFrame({'time':wind_proj.time.values})
    df = df.set_index('time')
    
    
    for model in wind_proj.model.values:
        constant = results[model]['constant']
        u2_coef = results[model]['u$^2$']
        v2_coef = results[model]['v$^2$']
        
        u2_data = wind_proj.u2.sel(model=model, drop=True).values
        v2_data = wind_proj.v2.sel(model=model, drop=True).values
        
        wind_contr = constant + (u2_coef * u2_data) + (v2_coef * v2_data)
        df[model] = wind_contr
        
    return df

In [ ]:
proj_119_np = wind_contr_proj_np(results_np, wind_119_np)
proj_126_np = wind_contr_proj_np(results_np, wind_126_np)
proj_245_np = wind_contr_proj_np(results_np, wind_245_np)
proj_370_np = wind_contr_proj_np(results_np, wind_370_np)
proj_585_np = wind_contr_proj_np(results_np, wind_585_np)

In [ ]:
import matplotlib.pyplot as plt

zos_119.zos.plot(hue='model', figsize=(9,3))
plt.title('ssp=119')
plt.ylabel('zos [cm]')
plt.xlabel('time [y]')
plt.axhline(color='k', linestyle='--', linewidth = 1)

In [ ]:
proj_119_np.plot(figsize=(9,3))
plt.title('ssp=119')
plt.ylabel('wind contribution [cm]')
plt.xlabel('time [y]')
plt.axhline(color='k', linestyle='--', linewidth = 1)

In [ ]:
proj_585_np.plot(figsize=(9,3))
plt.title('ssp=585')
plt.ylabel('wind contribution [cm]')
plt.xlabel('time [y]')
plt.axhline(color='k', linestyle='--', linewidth = 1)